<a href="https://colab.research.google.com/github/abdalrahmanmajzoub/Web-Scraping-Indeed-website/blob/main/Web_Scraping_Indeed_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing required libraries
!pip install bs4
!pip install requests

In [1]:
#import required libraries
import pandas as pd
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

#prepare the link
def get_url(postition , location):
  template = 'https://www.indeed.com/jobs?q={}&l={}'
  url = template.format(postition, location)
  print(url)
  return url

#get jobs records from html tags using BeautifulSoup library
def get_record(card):
  job_title= card.find('span' , {'title': True}).text.strip()
  company_name = card.find('span' , {'class':'companyName'}).text.strip()
  company_location = card.find('div' , {'class':'companyLocation'}).text.strip()
  job_summary = card.find('div' , {'class':'job-snippet'}).text.strip()
  job_link = 'https://www.indeed.com'+card.get('href')
  try:
    job_salary = card.find('span' , {'class':'salary-snippet'}).text.strip()
  except AttributeError:
    job_salary = ''
  
  record = (job_title,company_name,company_location,job_summary,job_salary,job_link)

  return record

def main(position,location):
  records = []
  print(position)
  print(location)
  url = get_url(position, location)

  while True:
      response = requests.get(url)
      print(response)
      soup = BeautifulSoup(response.text,'html.parser')
      cards = soup.find_all('a' , {'data-hide-spinner':"true"})
      for card in cards:
        record = get_record(card)
        print(record)
        records.append(record)
        print(len(records))
      try:
        url = 'https://www.indeed.com'+ soup.find('a', {'aria-label':'Next'}).get('href')
      except AttributeError:
        break
      
      with open('resault5.csv' , 'w' , newline='' , encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(['job_title','company_name','company_location','job_summary','job_salary','job_link'])
        writer.writerows(records)

  indeed_data = pd.DataFrame(columns=['job_title','company_name','company_location','job_summary','job_salary','job_link'])
  for x in records:
    j_title = x[0]
    j_cn = x[1]
    j_cl = x[2]
    j_s = x[3]
    j_sal = x[4]
    j_l = x[5]
    indeed_data = indeed_data.append({"job_title":j_title, "company_name":j_cn, "company_location":j_cl, "job_summary":j_s, "job_salary":j_sal, "job_link":j_l}, ignore_index=True)
  return indeed_data



In [2]:
df_data = main('Data Science', '')

Data Science

https://www.indeed.com/jobs?q=Data Science&l=
<Response [200]>
('Enterprise Fundamentals Data Product Manager', 'Bloomberg', 'New York, NY', "Our research data team is responsible for identifying, creating, and designing data solutions that leverage Bloomberg's proprietary data.", '', 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Ab67y_gTDH9tSaT0HLOcX4Q3W4UsL2WfVRyJV-qqww-S_SOAUpqHEOp-BmnzQ8WvSe4lC76o-Az25pzmGajHb0HIK0m_JqXVLLhn2i78Es-gt6Yni6aMzL1hadwWYoJ2tW7GsE8nA4cRVLJORlHPueY8a1wGM4bO4d1F8Rj0FnRQVLngMXHViDa6jTZZZtfB30w8sV-vgxMJlrYJs186ZoNXazet0NuwzYf-KVxB5hJt5y_fBiAFE0VVdg9GAbTULkeeGtpd7sLgz-m5yPwnGBCXyolqyHINTg8YNwikLuI1WVXOy3boegzMkppqlzr-Oocw2ICK2mUXf48RP9gT0nCG71-15FLNCELcRQSviNf44OdTaR0nLGDO4mQ3jNOcdGWokm8RBktCVmaWpNdoxntWLW75vdl1kg3k4u0MzdV9cTeJblZ2zlKxGXvX12a-JRan7E5pvzr49vgxK9yB5ZMtf90Y6ZikACmPcbFYzOz3MS6SnruAVOB2xqFSQS2QF-8UTKS0Kb2t21oEx4NLFTBRUPoYSrptaZHrU7qRqWib7zR641jRcjGYGCn8NDiuQ3PWtygbIgsLxFSiHWoXfz5VFOuscOiluhb7h9YHItMQDxCXhWO73p61hvoRyGWHKE96DYt

In [3]:
df_data.shape

(1012, 6)

In [4]:
df_data.columns

Index(['job_title', 'company_name', 'company_location', 'job_summary',
       'job_salary', 'job_link'],
      dtype='object')

In [5]:
df_data.head()

,job_title,company_name,company_location,job_summary,job_salary,job_link
0,Enterprise Fundamentals Data Product Manager,Bloomberg,"New York, NY",Our research data team is responsible for iden...,,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,"Analyst I/II, Data Science",Liberty Mutual Insurance,+1 locationRemote,Generate data-driven answers to open-ended str...,"$85,100 - $138,900 a year",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,Entry Level Data Scientist: 2022,IBM,"Research Triangle Park, NC 27709",You will implement and validate predictive mod...,,https://www.indeed.com/rc/clk?jk=d9dc2b2ff8863...
3,Remote Data Scientist (contract),Pfizer,Remote,Benchmark internal and public repositories for...,,https://www.indeed.com/rc/clk?jk=4e7590a713871...
4,Data Scientist/Analytics Entry Level Full Time...,Cummins Inc.,"Columbus, IN•Remote",Develops data profiles for data tables and ele...,,https://www.indeed.com/rc/clk?jk=44ab675983c2b...


In [6]:
print(max(df_data['job_salary']))

Up to $70 an hour
